In [2]:
import torch
from safetensors.torch import load_file
from transformers import ElectraConfig, ElectraForSequenceClassification

# 모델 경로 설정 (절대 경로 사용)
model_path = "C:/fintech_service/final_project/saved_model_KcELECTRA-base2/saved_model/"

# config.json 로드
config = ElectraConfig.from_pretrained(model_path)

# safetensors 파일 경로
safetensors_path = f"{model_path}/model.safetensors"

# 가중치 로드
state_dict = load_file(safetensors_path)

# Electra 모델 생성 및 가중치 적용
model = ElectraForSequenceClassification(config)
model.load_state_dict(state_dict)

# 가중치를 pytorch_model.bin으로 저장
pytorch_model_path = f"{model_path}/pytorch_model.bin"
torch.save(model.state_dict(), pytorch_model_path)

print(f"Electra 기반 모델이 {pytorch_model_path}에 저장되었습니다.")


Electra 기반 모델이 C:/fintech_service/final_project/saved_model_KcELECTRA-base2/saved_model//pytorch_model.bin에 저장되었습니다.


In [3]:
import torch
from transformers import ElectraConfig, ElectraForSequenceClassification

# 모델 경로 설정
model_path = "C:/fintech_service/final_project/saved_model_KcELECTRA-base2/saved_model/"
pytorch_model_path = f"{model_path}/pytorch_model.bin"

# config.json 로드
config = ElectraConfig.from_pretrained(model_path)

# Electra 모델 생성
model = ElectraForSequenceClassification(config)

# 저장된 가중치 로드
model.load_state_dict(torch.load(pytorch_model_path))
model.eval()  # 평가 모드로 전환

print("모델이 성공적으로 로드되었습니다!")

C:\Users\5-04\AppData\Local\Temp\ipykernel_11376\1363928214.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(pytorch_model_path))


모델이 성공적으로 로드되었습니다!


In [4]:
import os
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 1. 모델과 토크나이저 로드
model_path = 'C:/fintech_service/final_project/saved_model_KcELECTRA-base2/saved_model/'  # 모델 저장 경로
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # 모델을 평가 모드로 설정

# 2. 감성 분석할 CSV 파일 로드
input_file = "C:/fintech_service/final_project/data/뱅뱅막국수_combined_reviews_감정분석완료.csv"  # 분석할 파일 경로

if not os.path.exists(input_file):
    raise FileNotFoundError(f"파일이 존재하지 않습니다: {input_file}")

df = pd.read_csv(input_file)

# 3. 문자열이 아닌 값(NaN 등)을 제거
if 'Review' not in df.columns or 'sentiment' not in df.columns:
    raise ValueError("데이터프레임에 'Review' 또는 'sentiment' 컬럼이 없습니다.")

df = df.dropna(subset=['Review'])

# 4. 감성 분석 함수 정의
def predict_sentiment(text):
    if not isinstance(text, str):
        return "알 수 없음"  # 문자열이 아닌 경우 기본값 반환
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "긍정" if prediction == 1 else "부정"

# 5. 부정적으로 판별된 리뷰에 대해서만 재분석 수행
def reanalyze_if_negative(sentiment, text):
    if sentiment == "부정":
        # 부정적으로 판별된 경우에만 재분석 수행
        second_prediction = predict_sentiment(text)
        return second_prediction
    # 부정적이지 않은 경우 원래 값을 그대로 유지
    return sentiment

# 'sentiment' 컬럼이 부정적인 경우에만 재분석 수행하여 'Sentiment2'에 저장
df['Sentiment2'] = df.apply(lambda row: reanalyze_if_negative(row['sentiment'], row['Review']), axis=1)

# 6. 결과 파일 저장
base_name = os.path.splitext(os.path.basename(input_file))[0]  # 파일명만 추출 (확장자 제외)
output_dir = os.path.dirname(input_file)  # 기존 파일이 있는 폴더
output_file = os.path.join(output_dir, f"{base_name}_재분석_KcELECTRA-base2.csv")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df.to_csv(output_file, index=False, encoding="utf-8-sig")  # UTF-8 with BOM으로 저장
print(f"감성 분석 결과가 {output_file}에 저장되었습니다.")


감성 분석 결과가 C:/fintech_service/final_project/data\뱅뱅막국수_combined_reviews_감정분석완료_재분석_KcELECTRA-base2.csv에 저장되었습니다.
